In [1]:
import keras
from keras import backend as K
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
import keras.models as models
import keras.layers as layers
import tensorflow as tf
import _pickle as pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
import os
import json
import datetime
import pandas as pd

Using TensorFlow backend.


### Specify the model name and training data paths below

Separate directories for positive and negative examples of each class should be specified in train_dir_tp, and train_dir_fp, respectively

In [2]:

model_file_name = 'ResNet50_test' # name of trained model file
model_dir = '../data' # path to directory where model will be stored

train_dir_tp = '../data/train_tp' # directory with examples of true-positive spectrograms of each class
train_dir_fp = '../data/train_fp' # directory with examples of false-positive spectrograms of each class

num_classes = 24
input_shape = [224, 224, 3]
batch_size = 32
epochs = 5


### Run remaining cells to begin training

In [4]:
files = []
target = []
class_dict = dict()

for c, i in enumerate(sorted(os.listdir(train_dir_tp))):
    class_dict[c] = i
    for j in os.listdir(train_dir_tp+'/'+i):
        files.append(train_dir_tp+'/'+i+'/'+j)
        tmp = np.empty(num_classes)
        tmp[:] = np.nan
        tmp[c] = int(1)
        target.append(tmp)
        
for c, i in enumerate(sorted(os.listdir(train_dir_fp))):
    class_dict[c] = i
    for j in os.listdir(train_dir_fp+'/'+i):
        files.append(train_dir_fp+'/'+i+'/'+j)
        tmp = np.empty(num_classes)
        tmp[:] = np.nan
        tmp[c] = int(0)
        target.append(tmp)
        
df_train = pd.concat([pd.DataFrame({'filename':files}),pd.DataFrame(np.asarray(target))],axis=1)

print(len(df_train))
validation_indices = np.random.choice(range(len(df_train)), size=int(len(df_train)*0.1), replace=False)
df_validation = df_train.iloc[validation_indices]
df_train.drop(df_train.index[validation_indices], inplace=True)
print(len(df_train)+len(df_validation))
df_validation.reset_index(drop=True, inplace=True)
df_train.reset_index(drop=True, inplace=True)
df_train.head()



275560
275560


,filename,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,./training_data_13/rois_tp_links/10153/project...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,./training_data_13/rois_tp_links/10153/project...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,./training_data_13/rois_tp_links/10153/project...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,./training_data_13/rois_tp_links/10153/project...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,./training_data_13/rois_tp_links/10153/project...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
train_datagen = ImageDataGenerator(rescale=1/255.0)

test_datagen = ImageDataGenerator(rescale=1/255.0)

train_generator = train_datagen.flow_from_dataframe(df_train,
                                                    y_col=range(num_classes),
                                                    directory=None,
                                                    target_size=input_shape[:2],
                                                    batch_size=batch_size,
                                                    class_mode='raw')

validation_generator = test_datagen.flow_from_dataframe(df_test,
                                                        y_col=range(num_classes),
                                                        directory=None,
                                                        target_size=input_shape[:2],
                                                        batch_size=batch_size,
                                                        class_mode='raw')

def generator_wrapper(generator):
    for batch_x, batch_y in generator:
        yield (batch_x, np.row_stack(batch_y))



Found 248004 validated image filenames.
Found 27556 validated image filenames.


In [4]:
def masked_loss(y_true, y_pred):
    return K.mean(K.mean(K.binary_crossentropy(tf.where(tf.math.is_nan(y_true), tf.zeros_like(y_true), y_true),
                                        tf.multiply(y_pred, tf.cast(tf.logical_not(tf.math.is_nan(y_true)), tf.float32))), axis=-1))

In [7]:
#Load the ResNet50 model
ResNet50_conv = ResNet50(weights='imagenet', 
                         include_top=False, 
                         input_shape=input_shape)

for layer in ResNet50_conv.layers:
    layer.trainable = True

# Create the model
model = models.Sequential()
# Add the convolutional base model
model.add(ResNet50_conv)

model.add(layers.AveragePooling2D((7, 7)))

# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='sigmoid'))

 # Compile the model
optimizer = keras.optimizers.adam(lr=0.0001, decay=1e-7)
model.compile(loss=masked_loss, optimizer=optimizer, metrics=['accuracy'])

model.summary()


/home/genomics/Software/miniconda3/envs/sa/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
average_pooling2d_2 (Average (None, 1, 1, 2048)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                12312     
Total params: 24,649,112
Trainable params: 24,595,992
Non-trainable params: 53,120
_____________________________________

In [ ]:
model_json = model.to_json()
with open(model_path+'/'+model_file_name+'.json', "w") as json_file:
    json_file.write(model_json)
with open(model_path+'/'+model_file_name+'_classes.json', 'w') as f:
    json.dump(class_dict, f)
print('Saved model architecture')
    
model_history = model.fit_generator(train_generator,
                                steps_per_epoch = len(train_generator),
                                epochs = epochs,
                                validation_data = validation_generator,
                                validation_steps = len(validation_generator),
                                verbose = 1)
print('Saving model...')
model.save_weights(model_path+'/'+model_file_name+'.h5')
            

Saved model architecture
Epoch 1/1
 984/7751 [==>...........................] - ETA: 8:21:57 - loss: 0.0147 - accuracy: 0.1429